<a href="https://colab.research.google.com/github/neon-aiart/chirp-whisper-link/blob/main/chirp-whisper-link%20v4.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🐦 Chirping Whisper Link
APP_VERSION = '4.2'
# @markdown ---
model_type = "turbo" # @param ["tiny", "base", "small", "medium", "large-v3", "turbo"]
initial_prompt = "よろしくお願いします。" # @param {type:"string"}
mode = "Upload" # @param ["Upload", "YouTube"]
youtube_url = "Youtube URL" # @param {type:"string"}
language = "ja"
condition_on_previous_text = False  # @param {type:"boolean"}
# @markdown ---

import sys

# 1. ライブラリ準備
if 'whisper' not in sys.modules:
    print("📦 ライブラリをインストール中...")
    !pip install openai-whisper yt-dlp -q

import os, time, torch, gc, whisper
from datetime import datetime
from google.colab import files, output
from IPython.display import clear_output, Audio, display
import numpy as np

# ブラウザの言語設定(html lang)を取得
browser_lang = output.eval_js('document.documentElement.lang')

if 'ja' in browser_lang:
    language = "ja"
    os.environ['TZ'] = 'Asia/Tokyo'
else:
    language = "en"
    os.environ['TZ'] = 'UTC'

time.tzset() # タイムゾーンの設定を反映

# 2. モデルの自動ロード判別
if 'model' not in globals() or current_model_type != model_type:
    print(f"📦 Whisperモデル({model_type})を読み込み中... (数分かかります)")
    model = whisper.load_model(model_type)
    current_model_type = model_type
    print("✅ モデルのロードが完了しました。")
else:
    print(f"⚡ モデル({model_type})はロード済みです。即座に開始します。")

# 3. メモリ掃除
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 4. 入力ソースの選択
file_name = None
if mode == "YouTube":
    if youtube_url == "Youtube URL" or not youtube_url.startswith("http"):
        print("⚠️ 有効なYouTube URLを入力してください。")
    else:
        import yt_dlp
        print("📺 YouTubeから音声を抽出中...")
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': 'youtube_audio.%(ext)s',
            'noplaylist': True,
            'quiet': True # ログをスッキリさせる
        }
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(youtube_url, download=True)
                file_name = ydl.prepare_filename(info)
                if not os.path.exists(file_name):
                    # .webm などに変わっている可能性を考慮
                    file_name = info.get('requested_downloads', [{}])[0].get('filepath', file_name)
        except Exception as e:
            print(f"❌ YouTube動画のダウンロードに失敗しました: {e}")
            file_name = None # Ensure file_name is None if download fails
else:
    print("📂 ローカルファイルを選択してください")
    uploaded = files.upload()
    if uploaded:
        file_name = list(uploaded.keys())[0]

# 5. 文字起こし実行
if file_name:
    print(f"\n🚀 実行中: {file_name}")
    result = model.transcribe(
        file_name,
        initial_prompt=initial_prompt,
        language=language,
        condition_on_previous_text=condition_on_previous_text,
        verbose=False
    )

    # 保存
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    def format_timestamp(seconds):
        """秒数を SRT形式 (00:00:00,000) に変換"""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        millis = int((seconds - int(seconds)) * 1000)
        return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

    # SRT形式の作成
    srt_output = ""
    for i, segment in enumerate(result["segments"]):
        start = format_timestamp(segment["start"])
        end = format_timestamp(segment["end"])
        text = segment["text"].strip()
        srt_output += f"{i + 1}\n{start} --> {end}\n{text}\n\n"

    # 議事録を作る
    log_output = ""
    for segment in result["segments"]:
        start = format_timestamp(segment["start"])[:8] # ミリ秒をカットしてスッキリ
        text = segment["text"].strip()
        log_output += f"[{start}] {text}\n" # [00:01:23] こんにちは... という形式

    # output_name_txt = f"{os.path.splitext(file_name)[0]}_{timestamp}.txt"
    # with open(output_name_txt, "w", encoding="utf-8") as f:
    #     f.write(result["text"])

    output_name_srt = f"{os.path.splitext(file_name)[0]}_{timestamp}.srt"
    with open(output_name_srt, "w", encoding="utf-8") as f:
        f.write(srt_output)

    output_name_log = f"{os.path.splitext(file_name)[0]}_{timestamp}.txt"
    with open(output_name_log, "w", encoding="utf-8") as f:
        f.write(log_output)

    # 結果表示
    clear_output()
    print(f"✅ 完了！: {file_name}")
    print("-" * 30)
    print(result["text"][:300] + "...")
    print("-" * 30)

    # --- 完了通知音（きらめき音） ---
    sample_rate = 44100 # Hz

    # 音を構成する要素の数
    num_sparkles = 3
    # 各きらめき音の長さ
    sparkle_duration = 0.08 # 秒
    # 周波数の全体的な開始範囲と終了範囲
    base_start_frequency = 2000 # Hz
    base_end_frequency = 4000 # Hz (全体的に上昇するような効果)
    # 急速な減衰率
    decay_rate = 20

    all_data = []
    for i in range(num_sparkles):
        # 各きらめき音のタイムベクトル
        t_sparkle = np.linspace(0, sparkle_duration, int(sparkle_duration * sample_rate), endpoint=False)

        # 各きらめき音の開始周波数と終了周波数を計算
        # これにより、全体のきらめき音が上昇するシーケンスになります
        f_start_current_sparkle = base_start_frequency + (base_end_frequency - base_start_frequency) * (i / num_sparkles)
        f_end_current_sparkle = base_start_frequency + (base_end_frequency - base_start_frequency) * ((i + 1) / num_sparkles)

        # 各きらめき音内でわずかな上昇スイープを導入し、きらめき効果を強調
        # 実際の周波数は f_start_current_sparkle から f_end_current_sparkle まで変化します
        instantaneous_frequency = np.linspace(f_start_current_sparkle, f_end_current_sparkle, len(t_sparkle))

        # 波形を生成
        sparkle_wave = np.sin(2 * np.pi * instantaneous_frequency * t_sparkle)

        # 指数関数的減衰を適用
        envelope_sparkle = np.exp(-decay_rate * t_sparkle)
        sparkle_data = sparkle_wave * envelope_sparkle

        all_data.append(sparkle_data)

        # 各きらめき音の間に非常に短い無音を挿入（最後の音以外）
        # これにより、それぞれの音がはっきりと聞こえ、混ざり合うのを防ぎます
        if i < num_sparkles - 1:
            silence_duration = 0.02 # 秒
            silence = np.zeros(int(silence_duration * sample_rate))
            all_data.append(silence)

    data = np.concatenate(all_data)

    # ボリュームを正規化（クリッピング防止と音量調整）
    # データが空の場合はエラーにならないように処理
    data = data / np.max(np.abs(data)) * 0.8 if len(data) > 0 else np.array([0.0])

    display(Audio(data, rate=sample_rate, autoplay=True))

    # 6. ダウンロード
    print("📥 ファイルをダウンロードしています...")
    # files.download(output_name_txt)
    files.download(output_name_srt)
    files.download(output_name_log)

    print("\n" + "!"*40)
    print("⚠️ ATTENTION: PLEASE DISCONNECT RUNTIME")
    print("⚠️ 接続解除を忘れないでください！残り時間が削られます。")
    print("!"*40)

    # YouTube用の一時ファイル削除
    if mode == "YouTube" and file_name and os.path.exists(file_name):
        os.remove(file_name)
else:
    print("⚠️ 処理がキャンセルされました。")